# DD2424 Deep Learning in Data Science
## Assignment 4

### Imports

In [1]:
# import sys
# !{sys.executable} -m pip install numpy
# !{sys.executable} -m pip install matplotlib
# !{sys.executable} -m pip install tqdm

In [2]:
# import numpy as np
from data import *
from utils import *
from rnn_ import RNN

### Load data

In [3]:
data = load_and_process_text("goblet_book.txt")
book_data = data['book_data']
char_2_indices = data['char_2_indices']
indices_2_char = data['indices_2_char']

## Exercise 1
Implement and train a vanilla RNN

##### Synthesize text

In [4]:
rnn = RNN(80, 80, hidden_size=10, char_2_indices=char_2_indices, indices_2_char=indices_2_char)
output = rnn.generate()
output

"Wk7HUbat!JH'w:N4QQ^ent(A?"

##### Implement forward pass

In [5]:
seq_length = 25

x_chars = index_2_one_hot(np.array(chars_2_indices(book_data[:seq_length], char_2_indices)), 80)
y_chars = index_2_one_hot(np.array(chars_2_indices(book_data[1:seq_length + 1], char_2_indices)), 80)

In [6]:
loss = rnn.forward(x_chars, y_chars, np.zeros(shape=(10, 1)))

In [7]:
gradients = rnn.backward(x_chars, y_chars)

In [8]:
gradients_num = rnn.compute_gradients_num(x_chars, y_chars)

In [9]:
for grad in gradients:
    print(gradients[grad].shape, gradients_num[grad].shape)

(10, 10) (10, 10)
(10, 80) (10, 80)
(10, 1) (10, 1)
(80, 10) (80, 10)
(80, 1) (80, 1)


In [10]:
equal = True
threshold = 0

for grad in ['W']:
    equal = equal and np.allclose(gradients[grad], gradients_num[grad], rtol=1e-6, atol=1e-6)    
    threshold = max(threshold, np.max(np.abs(gradients[grad] - gradients_num[grad])))

print(equal)
print(threshold)

True
2.581783940282245e-10


In [11]:
gradients_num = rnn.compute_gradients(x_chars, y_chars)

AttributeError: 'RNN' object has no attribute 'compute_gradients'